In [1]:
import pandas as pd
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
import torch 

from maderapp.data.data_inference import MaderappDatasetInference

model_path = "maderapp-epoch=247-val_loss=0.05.ckpt"
model_name = "MobileNet"
num_classes = 27

In [2]:
metadata = pd.read_csv("../metadata-v3.csv", header=None)
class_names = sorted(metadata.iloc[:, 1].value_counts().index)
class_names2ids = {j: i for i, j in enumerate(class_names)}
class_ids2names = {j: i for i, j in class_names2ids.items()}


In [3]:
transformation = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],),
        ToTensorV2(),
    ]
)

In [4]:
model = torch.jit.load("opt_trace_mobileNet-v3.pt")
data_path = "../../maderapp_data/images/"
metadata = [str(path) for path in list(Path(data_path).glob("*.jpg"))]

print(metadata)

ds = MaderappDatasetInference(annotations_file=metadata, transform=transformation)
dl = DataLoader(ds, batch_size=3, shuffle=False, num_workers=4)

x, y = next(iter(dl))

y_pred = model(x)
class_pred = [class_ids2names[int(idx)] for idx in y_pred.argmax(dim=1)]

print(class_pred)

['../../maderapp_data/images/almendro.jpg', '../../maderapp_data/images/sapote.jpg', '../../maderapp_data/images/lanchan.jpg']
['Almendro', 'Sapote', 'Lanchan']


/home/anvelezec/anaconda3/envs/datascience/lib/python3.8/site-packages/torch/nn/modules/module.py:889: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:934.)
  result = self.forward(*input, **kwargs)


In [6]:
y_pred.max(dim=1)

torch.return_types.max(
values=tensor([1.0000, 0.9991, 0.9238]),
indices=tensor([ 0, 22,  9]))